In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import datetime
import statsmodels.formula.api as smf

In [2]:
os.chdir('D:\Janis\Desktop\研一上\资产定价（投资学）\A8')
funds = pd.read_sas('funds_for_assignment_8.sas7bdat')
mfret = pd.read_sas('MFret.sas7bdat')
ff = pd.read_csv('FF_monthly.csv')

In [3]:
ff.columns = ['date','mkt_rf','smb','hml','rf']
ff['date'] = pd.to_datetime(ff['date'],format='%Y%m')
ff['mkt'] = ff['mkt_rf']+ff['rf']
ff['date'] = ff['date']-datetime.timedelta(days=1)
mfret.columns = mfret.columns.map(lambda x:x.lower())

In [28]:
ret = pd.merge(mfret,funds, on = 'crsp_fundno')
ret = ret.rename(columns={'caldt':'date'})
ret = pd.merge(ret,ff,on = 'date')
ret = ret.dropna(subset=['mret'])
ret['obs'] = 1

In [29]:
reta = ret[ret['type']==b'A']
retb = ret[ret['type']==b'B']
retc = ret[ret['type']==b'C']
retd = ret[ret['type']==b'D']

In [49]:
def calc(ret):
    def f(group):
        group['nobs'] = group.loc[(group['date']>=datetime.datetime(1990,1,1))&(group['date']<=datetime.datetime(2008,12,31)),'obs'].sum()
        return group
    ret = ret.groupby('crsp_fundno').apply(f)
    print("-----------------Small Cap Value: Sample Analysis-----------------")
    print('sufficient:',ret.loc[ret['nobs']>=114,'crsp_fundno'].unique().shape[0])
    print('unsufficient:',ret.loc[ret['nobs']<114,'crsp_fundno'].unique().shape[0])
    ret = ret[ret['nobs']>=114]
    ret['mret'] = ret['mret']-ret['mkt']
    def des(x):
        print("mean:","%.5f"%x.mean(),
    "min:","%.5f"%x.min(),
    "max:","%.5f"%x.max(),
    "median:","%.5f"%x.median()
    )
    print("-----------------Small Cap Value: Average Excess Returns over Market Return-----------------")
    des(ret['mret'])
    def f2(group):
        model = smf.ols('mret~1+mkt_rf+smb+hml',group).fit()
        pa = model.params.to_frame()
        t = model.tvalues.to_frame()
        ans = pd.concat([pa,t],axis=1).reset_index()
        ans.columns=['coef','pa','t']
        ans['id'] = group.index[0]
        global result
        result = pd.concat([result,ans])
    global result
    temp = ret.groupby('crsp_fundno').apply(f2)
    result['crsp_fundno'] = list(ret['crsp_fundno'][result['id']])
    result = result[['crsp_fundno','coef','pa','t']]
    def anlys(x):
        mean = result.loc[result['coef']==x,'pa']
        t = result.loc[result['coef']==x,'t']
        des(mean)
        print('sign+', np.sum((t>2)!=0),'sign-',np.sum((t<-2)!=0))
    print("-----------------Small Cap Value: Analysis of Coefficients using FF Model-----------------")
    anlys('Intercept')
    anlys('mkt_rf')
    anlys('smb')
    anlys('hml')
    def f3(group):
        l = group['mret']-group['rf']
        group['sharp'] = l.mean()/group['mret'].std()
        return group
    ret = ret.groupby('crsp_fundno').apply(f3)
    sharp = ret['sharp'].drop_duplicates()
    print("-----------------Small Cap Value: Sharpe Ratio-----------------")
    des(sharp)

In [50]:
result = pd.DataFrame(columns=['coef','pa','t','id'])
calc(reta)

-----------------Small Cap Value: Sample Analysis-----------------
sufficient: 320
unsufficient: 1233
-----------------Small Cap Value: Average Excess Returns over Market Return-----------------
mean: -0.94111 min: -16.77129 max: 22.67294 median: -1.35691
-----------------Small Cap Value: Analysis of Coefficients using FF Model-----------------
mean: -0.27154 min: -0.49079 max: -0.18950 median: -0.24393
sign+ 0 sign- 320
mean: -0.99724 min: -1.00267 max: -0.99460 median: -0.99695
sign+ 0 sign- 320
mean: 0.00455 min: 0.00055 max: 0.02185 median: 0.00300
sign+ 44 sign- 0
mean: 0.00072 min: -0.00469 max: 0.01542 median: -0.00019
sign+ 15 sign- 0
-----------------Small Cap Value: Sharpe Ratio-----------------
mean: -0.26503 min: -0.36062 max: -0.18913 median: -0.24273


In [51]:
result = pd.DataFrame(columns=['coef','pa','t','id'])
calc(retb)

-----------------Small Cap Value: Sample Analysis-----------------
sufficient: 324
unsufficient: 2012
-----------------Small Cap Value: Average Excess Returns over Market Return-----------------
mean: -0.92891 min: -16.82647 max: 22.66132 median: -1.35327
-----------------Small Cap Value: Analysis of Coefficients using FF Model-----------------
mean: -0.28303 min: -0.48895 max: -0.18584 median: -0.27569
sign+ 0 sign- 324
mean: -0.99686 min: -1.00119 max: -0.99428 median: -0.99661
sign+ 0 sign- 324
mean: 0.00427 min: -0.00021 max: 0.01555 median: 0.00442
sign+ 27 sign- 0
mean: 0.00071 min: -0.00452 max: 0.01524 median: 0.00034
sign+ 11 sign- 0
-----------------Small Cap Value: Sharpe Ratio-----------------
mean: -0.26260 min: -0.37197 max: -0.18352 median: -0.24517


In [52]:
result = pd.DataFrame(columns=['coef','pa','t','id'])
calc(retc)

-----------------Small Cap Value: Sample Analysis-----------------
sufficient: 160
unsufficient: 1101
-----------------Small Cap Value: Average Excess Returns over Market Return-----------------
mean: -0.88682 min: -16.75804 max: 22.65014 median: -1.33725
-----------------Small Cap Value: Analysis of Coefficients using FF Model-----------------
mean: -0.25670 min: -0.49021 max: -0.18163 median: -0.24854
sign+ 0 sign- 160
mean: -0.99610 min: -1.00009 max: -0.99380 median: -0.99598
sign+ 0 sign- 160
mean: 0.00318 min: -0.00179 max: 0.01340 median: 0.00232
sign+ 16 sign- 0
mean: 0.00008 min: -0.00612 max: 0.01386 median: -0.00165
sign+ 6 sign- 0
-----------------Small Cap Value: Sharpe Ratio-----------------
mean: -0.25411 min: -0.36922 max: -0.17567 median: -0.24169


In [53]:
result = pd.DataFrame(columns=['coef','pa','t','id'])
calc(retd)

-----------------Small Cap Value: Sample Analysis-----------------
sufficient: 203
unsufficient: 278
-----------------Small Cap Value: Average Excess Returns over Market Return-----------------
mean: -0.94768 min: -12.90053 max: 22.62426 median: -1.37069
-----------------Small Cap Value: Analysis of Coefficients using FF Model-----------------
mean: -0.27917 min: -0.52067 max: -0.19007 median: -0.26323
sign+ 0 sign- 203
mean: -0.99739 min: -1.00193 max: -0.99218 median: -0.99718
sign+ 0 sign- 203
mean: 0.00219 min: -0.00156 max: 0.01062 median: 0.00126
sign+ 17 sign- 0
mean: -0.00101 min: -0.00629 max: 0.01297 median: -0.00227
sign+ 7 sign- 0
-----------------Small Cap Value: Sharpe Ratio-----------------
mean: -0.27075 min: -0.40219 max: -0.18540 median: -0.24959
